### Loading file 2000 pages
from assignment 2, working on small dataset, most of the pages are not the same as provided in
queries_train.json for each query

In [1]:
import os
from pathlib import Path
import pickle


pkl_file = "part15_preprocessed.pkl"
try:
    if os.environ["assignment_2_data"] is not None:
      pkl_file = Path(os.environ["assignment_2_data"])
except:
   Exception("Problem with one of the variables")
   
assert os.path.exists(pkl_file), 'You must upload this file.'
with open(pkl_file, 'rb') as f:
  pages = pickle.load(f)  


### Get Wikipedia Page Title
with given pages ids (list), the function generate a new list of the titles for each page id
not all pages exist, or for some reason doesn't have title, so we catch the error there, no bigi.

In [2]:
from urllib.request import urlopen
import json

def get_page_titles(pagesId):
  ''' Returns the title of the first, fourth, and fifth pages as ranked about 
      by PageRank.
  Returns:
  --------
    list of three strings.
  '''
  # YOUR CODE HERE
  def get_pageId_title(pageID):
    url_json_wiki = lambda x : f"https://en.wikipedia.org/w/api.php?action=query&inprop=url&pageids={x}&prop=info&format=json"   
    # store the response of URL
    response = urlopen(url_json_wiki(pageID)) 
    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())
    try:
      return data_json['query']['pages'][str(pageID)]['title']
    except:
      return pageID
    

  titles = [get_pageId_title(pageId) for pageId in pagesId]
  return titles

### Creation of indexes
creating indexes for:
- Body
- Title
- anchor

docs a set for all docs in our index, would be used later for intersection with pageViews to save only the relvent files for our index.

In [3]:
from Parser import process_wiki
body_idx, title_idx, anchor_idx, docs = process_wiki(pages, 'all_words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### ????

In [ ]:
from inverted_index import InvertedIndex

body_idx2 = InvertedIndex.read_index('body_indices', 'all_words')
# title_idx = InvertedIndex.read_index('title_index', 'all_words')
# anchor_idx = InvertedIndex.read_index('anchor_index', 'all_words')


### Tokenizer
an basic tokenizer, later on will add classification of:
- Bold words, need to understand if its an html or wikimedia syntax
- Dates? years?
- U.S.A == USA to get consistent and reasonable queries

and more....

In [4]:
import re
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('stopwords')


RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]    
    return list_of_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading Dataset
Given Queries and relevant Documents for each query.
we load and later on will test the Measures.

In [5]:
import json
with open('queries_train.json') as f:
    data = json.load(f)
# (q_id : (query_text, [relvent docs_ids]))
test_data = {q_id: d for q_id, d in enumerate(data.items())}
queries = {q_id: qf[0] for q_id, qf in test_data.items()}


### Search Functions 
So far done:
#### Minimum Requierments:
- Cosine Similarity on body
- Binary Similarity on title
- Binary Similarity on anchor
- Page Views ranked 

### TODO
- PageRank, assignment 3 contain the GraphQL implementation
- Build adapters for all functions to work using PySpark


In [ ]:
from search import Search
from cosineSimilarity import Cosine_Similarity
from binarySimilarity import Binary_Similarity

queries_tokenize = {q_id: tokenize(q) for q_id, q in queries.items()}
search = Search()
tfidf_queries_score_train_bsS_title = search.get_topN_score_for_queries(queries_tokenize, title_idx, N=30, score=Binary_Similarity)
tfidf_queries_score_train_bsS_anchor = search.get_topN_score_for_queries(queries_tokenize, anchor_idx, N=30, score=Binary_Similarity)
tfidf_queries_score_train_cosS_body = search.get_topN_score_for_queries(queries_tokenize, body_idx, N=30, score=Cosine_Similarity)

## Track and don't Delete previous Datasets, Measure the Engine Improvments
### Writing the results from Similarity function
Writing to Excel files on different sheets the results
!!! Warnning !!! take alot of time to run so next box offer faster display
but inconvinent. (15 minutes N=30)

In [ ]:
import pandas as pd

with pd.ExcelWriter("Output.xlsx") as writer:
    pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_bsS_title.items()}, \
     orient='index').to_excel(writer, sheet_name='Binary_Similarity_title', engine='xlsxwriter')
    pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_bsS_anchor.items()}, \
     orient='index').to_excel(writer, sheet_name='Binary_Similarity_anchor', engine='xlsxwriter')
    pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_cosS_body.items()}, \
     orient='index').to_excel(writer, sheet_name='Cosine_Similarity_body', engine='xlsxwriter')

### Writing to CSV
Much faster then Writing to an Excel File, but hard to read and understand

In [ ]:
from pathlib import Path
import pandas as pd
dirname = os.path.join(os.getcwd(), 'Data')
os.makedirs(dirname, exist_ok=True)

pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_bsS_title.items()}, \
    orient='index').to_csv(path_or_buf=os.path.join(dirname, 'Binary_Similarity_title.csv'), index=False)
pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_bsS_anchor.items()}, \
    orient='index').to_csv(path_or_buf=os.path.join(dirname, 'Binary_Similarity_anchor.csv'), index=False)
pd.DataFrame.from_dict({queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_cosS_body.items()}, \
    orient='index').to_csv(path_or_buf=os.path.join(dirname, 'Cosine_Similarity_body.csv'), index=False)

### Page Views Similarity
with given file of 2021-08 current views, Intersect the files in index with related views.
Sort & Slice(N=30)
Later on write to Disk as pkl OR find way to keep on memory.

In [ ]:
from pageViews import PageViews
N = 30
pageViews_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
docs_views = PageViews(pageViews_path, docs)
print(sorted(list(docs_views.items()), key=lambda x: x[1], reverse=True)[:N])


In [7]:
from search import Search
from cosineSimilarity import Cosine_Similarity
from binarySimilarity import Binary_Similarity

queries_tokenize = {q_id: tokenize(q) for q_id, q in queries.items()}
search = Search()
tfidf_queries_score_train_cosS_body = search.get_topN_score_for_queries(queries_tokenize, body_idx, N=30, score=Cosine_Similarity)

{queries[q_id] : list(map(get_page_titles, results)) for q_id, results in \
    list(tfidf_queries_score_train_cosS_body.items())[:5]}

[0.33544886 0.86728093 0.52826982]
[0.72895648 0.49157933 0.25559556]
[0.64824571 0.        ]
[0.]
[0.60369555 1.17328514]
[0.]
[0.4024637  0.71822825 0.32968631]
[0.70824051 0.26842282 0.25559556]
[1.52996078]
[0.4024637  0.53108087 0.71822825]
[2.69875279]
[1.17328514 0.89783141]
[2.82369152]
[0.38339334 0.68826675]
[0.9998914  0.88466694]
[2.69875279]
[0.6525888  0.94291972]
[0.83367217 1.12971005]
[0.60369555 0.69616389]
[2.82369152]
[0.60369555 0.        ]
[1.56841903]
[1.06236077 0.        ]
[0.52280634 0.4811601  0.72895648]
[0.25559556 0.4588445  0.89958426]
[1.3493764  1.26133077]
[0.]
[0.33544886 0.33917052 0.41463598]
[0. 0.]
[1.41184576 1.03518194]


{'best marvel movie': [['Howard Nostrand', 0.96515],
  ['Ondine (ballet)', 0.82262],
  ['Greek diacritics', 0.81094],
  ['Viscoplasticity', 0.81094],
  ['Modulating retro-reflector', 0.81094],
  ['Jaguar 420 and Daimler Sovereign (1966–1969)', 0.58513],
  ['Speaking in Strings', 0.58513],
  ['Loosely Tight', 0.58513],
  ['120 (film)', 0.58513],
  ['EA Bright Light', 0.58513],
  ['Tomoaki Maeno', 0.58513],
  ['S. Darko', 0.58245],
  ['Roberto Ampuero', 0.55201],
  ['Tony Duquette', 0.55201],
  ['Mannhai', 0.55201],
  ['Ariel Winter', 0.55201],
  ['Yellow Bird (company)', 0.52867],
  ['Tatsuhisa Suzuki', 0.51722],
  ['Nathan Furst', 0.51284],
  ['Warehouse 13', 0.49395],
  ['2008 California Proposition 4', 0.49395],
  ['Conanicut Island Light', 0.49395],
  ['Conimicut Light', 0.49395],
  ['Warwick Light', 0.49395],
  ['Sci-Fi on the Rock', 0.49395],
  ['Buffalo Sharks', 0.49395],
  ['Big Noise from Winnetka', 0.49395],
  ['Kenny Wollesen', 0.49395],
  ['John Hilliard', 0.49395],
  ['List